<a href="https://colab.research.google.com/github/michaelrochon1/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-05-14 20:51:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.85MB/s    in 0.2s    

2023-05-14 20:51:58 (4.85 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read the Database 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...|2015-08-31 00:00:00|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...| 

In [12]:
# Filter the dataset for total_votes being greater than or equal to 20
votes_20plus_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
votes_20plus_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|                Y|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|                Y|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|                Y|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|                Y|
| RNYLPX611NB7Q|          5|            0|          0|   N|                Y|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|                Y|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|                Y|
|R18WWEK8OIXE30|          5|            0|          0|   N|                Y|
|R1EGUNQON2J277|          1|            0|          0|   N|                Y|
|R21Z4M4L98CPU2|          5|            0|          0|   N|     

In [13]:
votes_20plus_df = vine_df.filter("total_votes >= 20")
votes_20plus_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R2JMJOWGM7V7NX|          4|           20|         20|   N|                N|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|     

In [15]:
# Filter the data for helpful_votes/total_votes is >= 50%
topvotes_df = votes_20plus_df.filter(votes_20plus_df["helpful_votes"]/votes_20plus_df["total_votes"]>=0.5)
topvotes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R2JMJOWGM7V7NX|          4|           20|         20|   N|                N|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|     

In [16]:
# Total number of reviews
topvotes_df.count()

355

In [18]:
# How many Vine subscribers account for reviews
review_df = topvotes_df.filter(topvotes_df["vine"] == "Y")
review_df.count()

0

In [19]:
review_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [20]:
# How many Vine subscribers did not leave a review
noreview_df = topvotes_df.filter(topvotes_df["vine"] == "N")
noreview_df.count()

355

In [21]:
noreview_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36PQ9D8L2AAOH|          5|           30|         32|   N|                Y|
|R23GHBE86II0SK|          5|           18|         21|   N|                Y|
|R18KVPY0TO33BM|          5|          241|        248|   N|                Y|
|R2ZB78BDM8BRCF|          1|           99|        113|   N|                Y|
|R2JMJOWGM7V7NX|          4|           20|         20|   N|                N|
|R10ORTN0I3G5B9|          1|           19|         26|   N|                Y|
|R1D1VPJBTP1WG8|          1|           38|         51|   N|                Y|
|R3Q3ULUTULAQYH|          1|           22|         29|   N|                Y|
|R1SHEELXA1IXQP|          5|           45|         57|   N|                Y|
|R1IQJ99JOPJOE4|          5|           73|         94|   N|     

In [22]:
# Number of five star reviews
fivestar_df = topvotes_df.filter(topvotes_df["star_rating"]== 5)
fivestar_df.count()

90

In [23]:
# Percent of reviews were 5*
fivestar_df.count()/topvotes_df.count()*100

25.352112676056336

In [27]:
# Percentage of 5* reviews from subscribers
fivestar_df.filter(fivestar_df["verified_purchase"] == "Y").count()/topvotes_df.filter(topvotes_df["verified_purchase"] == "Y").count()*100

23.076923076923077

In [29]:
# Percentage of 5* reviews from non subscribers
fivestar_df.filter(fivestar_df["verified_purchase"] == "N").count()/topvotes_df.filter(topvotes_df["verified_purchase"] == "N").count()*100

31.57894736842105